imports

In [2]:
import tensorflow as tf
import json
import tensorflow_datasets as tfds
from tensorflow.keras.utils import pad_sequences

opening the json file

In [3]:
with open("sarcasm.json", 'r') as f:
    datastore = json.load(f)

In [4]:
# Non-Sarcastic headline

print(datastore[0])

# Sarcastic headline

print(datastore[20000])

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5', 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers", 'is_sarcastic': 0}
{'article_link': 'https://www.theonion.com/pediatricians-announce-2011-newborns-are-ugliest-babies-1819572977', 'headline': 'pediatricians announce 2011 newborns are ugliest babies in 30 years', 'is_sarcastic': 1}


collecting the headlines because those are the string inputs that will be preprocessed into numeric features.

In [5]:
sentences = [item['headline'] for item in datastore]

pre-processing headlines

In [6]:
vectorize_layer = tf.keras.layers.TextVectorization()

vectorize_layer.adapt(sentences)

post_padded_sequences = vectorize_layer(sentences)

I0000 00:00:1754319649.094883   40617 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5409 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


checking results

In [7]:
index = 69
print(f'sample headline: {sentences[index]}')
print(f'padded sequence: {post_padded_sequences[index]}')
print()

print(f'shape of padded sequences: {post_padded_sequences.shape}')

sample headline: ryan lochte apologizes for behavior in rio
padded sequence: [ 389 6335 1028    6 2039    5 4286    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]

shape of padded sequences: (26709, 39)


using ragged tensor function to generate pre-padded sequences.

In [8]:
vectorize_layer = tf.keras.layers.TextVectorization(ragged=True)

vectorize_layer.adapt(sentences)

In [9]:
ragged_sequences = vectorize_layer(sentences)

In [10]:
index = 69
print(f'sample headline: {sentences[index]}')
print(f'padded sequence: {ragged_sequences[index]}')
print()

print(f'shape of padded sequences: {ragged_sequences.shape}')

sample headline: ryan lochte apologizes for behavior in rio
padded sequence: [ 389 6335 1028    6 2039    5 4286]

shape of padded sequences: (26709, None)


In [11]:
pre_padded_sequences = pad_sequences(ragged_sequences.numpy())

pre_padded_sequences[2]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,   140,   825,
           2,   813,  1100,  2048,   571,  5057,   199,   139,    39,
          46,     2, 13050], dtype=int32)